In [ ]:
# If first time using gradio, try to run this
pip install gradio

In [1]:
import numpy as np
import pandas as pd

import sklearn 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

# Import plotting libraries
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
import optuna
import sklearn.model_selection
import sklearn.preprocessing

#user interface
import gradio as gr

# Set larger fontsize for all plots
matplotlib.rcParams.update({'font.size': 18})
from IPython.display import clear_output

### Load data

In [4]:
df = pd.read_csv('bg_struct.csv', index_col=0)

In [5]:
df.describe()

,bandgap,cell_length_a,cell_lentgh_b,cell_lentgh_c,cell_angle_alpha,cell_angle_beta,cell_angle_gamma
count,16335.000000,16335.000000,16335.000000,16335.000000,16335.000000,16335.000000,16335.000000
mean,2.413936,7.756010,8.213617,11.564255,84.577062,87.422030,87.855174
std,0.373129,3.181366,2.887233,9.437813,18.476919,16.642053,22.486535
min,1.800200,2.363000,2.363000,2.661370,5.200300,5.397478,5.397478
25%,2.088000,5.609011,6.203360,7.542867,74.139139,88.470574,86.084645
50%,2.389300,7.131163,7.760993,9.836234,90.000000,90.000000,90.000000
75%,2.728300,9.018695,9.606510,12.994967,90.000000,90.000000,91.056782
max,3.099700,38.477274,40.417878,211.425839,168.624134,168.624134,167.256937


#### Define x and y

In [7]:
X = df[['cell_length_a','cell_lentgh_b', 'cell_lentgh_c',
        'cell_angle_alpha', 'cell_angle_beta', 'cell_angle_gamma']]
y = df['bandgap']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Train the decision tree model to predict bandgap

In [30]:
# Define the decision tree regressor object
regr = DecisionTreeRegressor(max_depth=9, min_samples_split=37, min_samples_leaf=4,
                             splitter='best', min_weight_fraction_leaf=0.00952179564895864)

#Fit to the training data
regr = regr.fit(X_train, y_train)

### Train the random forest model to predict bandgap

In [29]:
# Define a random forest regressor object
rand = RandomForestRegressor(n_estimators=99, max_depth=23, min_samples_split=14, 
                              min_samples_leaf=3, min_weight_fraction_leaf=2.9893745932052474e-05)

# Fit to your training data
rand = rand.fit(X_train, y_train)

### Use gradio to create user interface to show the predict bandgap value

In [31]:
def predict_bandgap(length_a, length_b, length_c, angle_alpha, angle_beta, angle_gamma, method):
    # convert the crystal lattice constant to a list
    test = [length_a, length_b, length_c, angle_alpha, angle_beta, angle_gamma]
    # convert the list to the dataframe
    test_point = pd.DataFrame([test], columns=['cell_length_a','cell_lentgh_b', 'cell_lentgh_c',
                                             'cell_angle_alpha', 'cell_angle_beta', 'cell_angle_gamma'])
    
    if method=='Random Forest':
        return rand.predict(test_point)
    else:
        return regr.predict(test_point)
    
    
#create input and output objects
#input object1
input1 = gr.inputs.Textbox(placeholder='Enter Crystal Lattice Length a')
input2 = gr.inputs.Textbox(placeholder='Enter Crystal Lattice Length b')
input3 = gr.inputs.Textbox(placeholder='Enter Crystal Lattice Length c')
input4 = gr.inputs.Textbox(placeholder='Enter Crystal Lattice Angle Alpha')
input5 = gr.inputs.Textbox(placeholder='Enter Crystal Lattice Angle Beta')
input6 = gr.inputs.Textbox(placeholder='Enter Crystal Lattice Angle Gamma')
input7 = gr.inputs.Radio(['Random Forest', 'Decision Tree'])

#output object
output = gr.outputs.Textbox()

#create interface
gui = gr.Interface(fn=predict_bandgap,
                  inputs=[input1,input2,input3,input4,input5,input6,input7],
                  outputs=[output])
gui.launch()

Running on local URL:  http://127.0.0.1:7863/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7fca9c788750>,
 'http://127.0.0.1:7863/',
 None)